<a href="https://colab.research.google.com/github/raisachatterjee/first/blob/master/resnet_cytology.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
import torch
import torchvision
import torchvision.transforms as transforms
from torchvision import datasets, transforms ,models
import helper


In [9]:
data_transforms = transforms.Compose([
                    transforms.Resize((224,224)),
                    transforms.ToTensor()])
train_set= datasets.ImageFolder(root="/content/drive/MyDrive/Dataset/train" , transform=data_transforms) # thank you
train_loader = torch.utils.data.DataLoader(train_set, batch_size=32, shuffle=True, num_workers=2)

val_set= datasets.ImageFolder(root="/content/drive/MyDrive/Dataset/validation" , transform=data_transforms) # thank you
val_loader = torch.utils.data.DataLoader(val_set, batch_size=32, shuffle=True, num_workers=2)


testset= datasets.ImageFolder(root="/content/drive/MyDrive/Dataset/test" , transform=data_transforms) # thank you
testloader = torch.utils.data.DataLoader(testset, batch_size=32, shuffle=True, num_workers=2)





classes=('metaplastic','parabasal','kolicytic','superficial','dyskeratotic')

device = torch.device("cuda:0" if torch.cuda.is_available()
                               else "cpu")

In [10]:
model = models.resnet34(pretrained=True)

num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 5)
model = model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

Downloading: "https://download.pytorch.org/models/resnet34-333f7ec4.pth" to /root/.cache/torch/hub/checkpoints/resnet34-333f7ec4.pth


In [11]:
for epoch in range(25):
    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        inputs, labels = data
        inputs = inputs.to(device)
        labels = labels.to(device)
        
        optimizer.zero_grad()
        
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
    print(running_loss)
print('Finished Training')

43.86359679698944
14.71680250018835
8.950152307748795
5.922672763466835
3.6850313586182892
2.965075606945902
1.7780593940988183
1.5027072185184807
1.1597185565624386
0.8459855736000463
0.8460624612635002
0.8523797641973943
0.5364238606998697
0.46713090571574867
0.37495383265195414
0.5050561217940412
0.42775590607197955
0.30586355667037424
0.26296526310034096
0.2381653346819803
0.174938128242502
0.2766949817305431
0.3255910008447245
0.38128931206301786
0.23087945465522353
Finished Training


In [14]:
class_correct = list(0. for i in range(5))
class_total = list(0. for i in range(5))
with torch.no_grad():
    for i, data in enumerate(val_loader, 0):
            inputs, labels = data
            inputs = inputs.to(device)
            labels = labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            c = (predicted == labels).squeeze()
            for i in range(5):
                label = labels[i]
                class_correct[label] += c[i].item()
                class_total[label] += 1
for i in range(5):
    print('Accuracy of %5s : %2d %%' % (
        classes[i], 100 * class_correct[i] / class_total[i]))

Accuracy of metaplastic : 96 %
Accuracy of parabasal : 80 %
Accuracy of kolicytic : 95 %
Accuracy of superficial : 100 %
Accuracy of dyskeratotic : 100 %


In [15]:
class_correct = list(0. for i in range(5))
class_total = list(0. for i in range(5))
with torch.no_grad():
    for i, data in enumerate(testloader, 0):
            inputs, labels = data
            inputs = inputs.to(device)
            labels = labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            c = (predicted == labels).squeeze()
            for i in range(5):
                label = labels[i]
                class_correct[label] += c[i].item()
                class_total[label] += 1
for i in range(5):
    print('Accuracy of %5s : %2d %%' % (
        classes[i], 100 * class_correct[i] / class_total[i]))

Accuracy of metaplastic : 90 %
Accuracy of parabasal : 100 %
Accuracy of kolicytic : 100 %
Accuracy of superficial : 95 %
Accuracy of dyskeratotic : 100 %
